# ID3 Algorithm

Method by which a decision tree can be generated

Write a function that computes the entropy of a set $S$ with ${n}_{pos}$ positive observations and ${n}_{neg}$ negative
observations.

The entropy of a set $S$ is given by equation:


\begin{equation*}
 entropy(S) = - \sum_{c=1}^C p_c log_2 p_c,
\end{equation*}

where $C$ is the number of classes (2 in our case since we have a positive and a negative class) and $p_c$ is the frequency of class $c$ in the set $S$.

In [1]:
import pandas as pd
from math import log

In [2]:
#2017, 2018 and 2019 Question Example
df = pd.DataFrame({"G":[0,1,1,0,0,0,1,1,0,1], "A":[1,1,3,2,2,1,2,3,3,1], "B":[1,1,1,0,1,0,1,1,0,1], "T":[1,1,0,0,1,0,1,1,0,1]})

In [3]:
from collections import Counter
import math
def entropy(S):
    entropy = 0
    n = len(S)
    frequencies = Counter(S)
    
    for key in frequencies.keys():
        pc = frequencies[key]/n
        entropy += (pc)*math.log(pc, 2)
    
    return -1*entropy

In [4]:
for attr in df.columns:
    print("Attribute: {}\nEntropy: {}\n".format(attr, entropy(df[attr])))


Attribute: G
Entropy: 1.0

Attribute: A
Entropy: 1.5709505944546684

Attribute: B
Entropy: 0.8812908992306927

Attribute: T
Entropy: 0.9709505944546686



Write a function that takes as input a set $S$ of examples and an feature $A$ from these examples,
and calculates the Information Gain, denoted as $gain(S,A)$.

The Information Gain is defined as:

\begin{equation*}
 gain(S,A) = entropy(S) - \sum_{i} \frac{|S_{u_i}|}{|S|} entropy(S_{u_i}),
\end{equation*}

where $S_{u_i}$ is the subset of examples from $S$ whose feature $A$ is equal to the $i^{th}$ possible value that $A$ can take. For example, the feature _Temperature_ can take on of three possible values: _Hot,_ _Mild,_ or _Cool;_ hence $S_{u_i}$ = $S_{Hot}$ denotes all the examples that have the _Hot_ value for the _Temperature_ feature. The
$\sum_{i}$ summation over $i$ indicates we are summing over all the possible values $u_i$ of feature $A$. Finally, $|S|$ and $|S_{u_i}|$ denote the cardinality of the set $S$ and subset $S_{u_i}$ respectively (cardinality = number of elements of the set).

In [5]:
def info_gain(df, feature, target):
#     Work out entropy of target column
#     Work out the entropy of the target column with respect to the different values in the feature column
    f_vals = df[feature].unique() # Ui
    gain = entropy(df[target]) # ent(S)
    t_size = len(df[target]) # |S|
    for key in f_vals:
        sui = df[df[feature] == key][target] # Sui
        gain -= (len(sui)/t_size)*entropy(sui)
    return gain

Estimate the Information Gain of all the attributes. Which one would you choose for the root node (or first decision stump) of your decision tree?

In [6]:
for attr in df.columns:
    print("Attribute: {}\nInfo Gain: {}\n".format(attr, info_gain(df, attr, "T")))

Attribute: G
Info Gain: 0.12451124978365313

Attribute: A
Info Gain: 0.09546184423832182

Attribute: B
Info Gain: 0.5567796494470394

Attribute: T
Info Gain: 0.9709505944546686



### Decision Tree induction

Algorithm 1 below shows the pseudo-code of the ID3 Decision Tree algorithm, which is a recursive algorithm that computes a decision tree based on a metric; in the case, the metric is Information Gain. Implement the ID3 algorithm and test it with the _play_tennis_ dataset; in other words, compute the training error. Note that the column _Day_ is not an attribute, it is only used to organize the examples. Also note that in Algorithm 1, Observations = Examples, Targets = Labels, and Attributes = Features.

## Algorithm 1:  ID3 Decision Tree Algorithm

**ID3** (Observations, Targets, Attributes)<br>

**if** all Observations are class +1 **then**<br>
&emsp; **return** single-node tree Root with label +1<br>
**if** all Observations are class -1 **then**<br>
&emsp; **return** single-node tree Root with label -1<br>
**if** Attributes is empty **then**<br>
&emsp; **return** single-node tree Root with label of most common value in Targets<br>
**else**<br>
&emsp;**begin**<br>
&emsp;$A$ $\leftarrow$ best attribute from Attributes (highest Information Gain)<br>
&emsp;The decision attribute for Root $\leftarrow$  $A$<br>
&emsp;**for** each possible vale $u_i$ of $A$: **do**<br>
&emsp;&emsp;Add a new tree branch below Root for $A = u_i$<br>
&emsp;&emsp;$S_{u_i}$ $\leftarrow$  Subset of Observations with $A = u_i$<br>
&emsp;&emsp;**if** $S_{u_i}$ is empty **then**<br>
&emsp;&emsp;&emsp;Add leaf node with label the most common value in Targets<br>
&emsp;&emsp;**else**<br>
&emsp;&emsp;&emsp;Add below branch **ID3**($S_{u_i},$  Targets, Attributes - {$A$})<br>
&emsp; **end**<br>
**return** Root<br>

In [7]:
class Node:
    def __init__(self, label):
        self.children = []
        self.label = label
    def add_child(self, child):
        self.children.append(child)

In [8]:
def ID3(S, target):
    if set(S[target].values) == set([1]):
        return Node(1)
    if set(S[target].values) == set([0]):
        return Node(0)
    if list(S.columns.drop([target])) == []:
        return Node(S[target].mode()[0])
    else:
        attr_per_info_gain = {info_gain(S, attr, target):attr for attr in sorted(S.columns.drop(target), reverse=True)}
        best_attr = attr_per_info_gain[max(attr_per_info_gain.keys())]
        root = Node(best_attr)
        for u in S[best_attr].unique():
            child = Node(u)
            root.add_child(child)
            new_S = S[S[best_attr] == u]
            if list(new_S[target]) == []:
                child.add_child(Node(S[target].mode()[0]))
            else:
                child.add_child(ID3(new_S.drop([best_attr], axis = 1),target))
    return root
            
            
        

In [9]:
tree = ID3(df,"T")

In [15]:
print(tree.label)
def print_tree(tree, tabs):
    for child in tree.children:
        print("{} {}".format(tabs*"   |", child.label))
        print_tree(child, tabs+1)
print_tree(tree, 1)

B
   | 1
   |   | A
   |   |   | 1
   |   |   |   | 1
   |   |   | 3
   |   |   |   | G
   |   |   |   |   | 1
   |   |   |   |   |   | 0
   |   |   | 2
   |   |   |   | 1
   | 0
   |   | 0
